# Machine Configuration

In [62]:
OMNETPP_PATH = "C:\\omnet\\omnetpp-5.5.1\\"
SDN4CORE_PATH = "/C/Users/Timo/Workspace/omnet/SDN4CoRE/"

In [63]:
import glob
import pandas as pd
import numpy as np
import ntpath
import matplotlib.pyplot as plt
import os
%matplotlib inline

## Helpers:
def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

# Scavetool -- parse filtered result files into csv

In [64]:

RESULTS_DIR = SDN4CORE_PATH + "examples/papers/vtc2023spring/scaling/"
MODULE_FILTER = "**.publishers**.middleware.**"
resultfiles = glob.glob("./results/SDN_Switches_Study-S=1,N=1-#0.sca")
for resultfile in resultfiles:
    command = OMNETPP_PATH + "tools\win64\\msys2_shell.cmd -mingw64 -ic \"echo " + resultfile.replace("\\","/").replace(".sca",".*") + "; scavetool x " + RESULTS_DIR + resultfile.replace("\\","/") + " " + RESULTS_DIR + resultfile.replace("\\","/").replace(".sca",".vec") + " -T v -f \'module(" + MODULE_FILTER + ")\'" + " -o " + RESULTS_DIR + resultfile.replace("\\","/").replace(".sca",".csv") + "; sleep 5\""
    print(command)
    os.environ["HOME"] = OMNETPP_PATH
    os.system(command)
time.sleep(60)


C:\omnet\omnetpp-5.5.1\tools\win64\msys2_shell.cmd -mingw64 -ic "echo ./results/SDN_Switches_Study-S=1,N=1-#0.*; scavetool x /C/Users/Timo/Workspace/omnet/SDN4CoRE/examples/papers/vtc2023spring/scaling/./results/SDN_Switches_Study-S=1,N=1-#0.sca /C/Users/Timo/Workspace/omnet/SDN4CoRE/examples/papers/vtc2023spring/scaling/./results/SDN_Switches_Study-S=1,N=1-#0.vec -T v -f 'module(**.publishers**.middleware.**)' -o /C/Users/Timo/Workspace/omnet/SDN4CoRE/examples/papers/vtc2023spring/scaling/./results/SDN_Switches_Study-S=1,N=1-#0.csv; sleep 5"


# Parse Results from csv

In [69]:
CSV_FILE = "./results/SDN_Switches_Study-S=1,N=1-#0.csv"
study = pd.read_csv(CSV_FILE, converters = {
    'attrvalue': parse_if_number,
    'binedges': parse_ndarray,
    'binvalues': parse_ndarray,
    'vectime': parse_ndarray,
    'vecvalue': parse_ndarray})
filtered = study[(study.type=="vector") & (study.name=="remotes:vector(constant1)")]
lastRemote[CSV_FILE]["MAX"] = filtered["vectime"].max()
lastRemote[CSV_FILE]["MIN"] = filtered["vectime"].min()
lastRemote

{'./results/SDN_Switches_Study-S=1,N=1-#0.csv': {'MAX': array([0.00949484]),
  'MIN': array([0.00949484])}}